In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import numpy as np
import datetime as dt
import math as math
pd.set_option('display.max_columns', None)

In [2]:
firmLoc = pd.read_csv('Main_Firm_No Loc_Fin v160225.csv')

In [3]:
# firm = firm.drop(["Unnamed: 0"], axis = 1)
# firm

In [4]:
# loc = pd.read_excel('Prowess IQ - Location Data.xlsx', skiprows = 5)

In [5]:
# loc = loc.drop( ["Registrar Address", "Company Name"], axis = 1).dropna(how = "all")
# loc

In [6]:
# loc["Office"] = np.where( pd.isnull(loc["Head Office"]), loc["Corporate Office"], loc["Head Office"])
# loc["Office"] = np.where( pd.isnull(loc["Office"]), loc["Registered Office"], loc["Office"])

In [7]:
# loc2 = loc.drop( ["Corporate Office", "Head Office", "Registered Office"], axis = 1).dropna(how = "all")
# loc2

In [8]:
# firmLoc = firm.merge(loc2, on = ["Prowess company code"], how = "inner")

In [9]:
firmLoc

,Unnamed: 0,FirmID,Symbol,AsOnDate,TotDirCount,LnBoardSize,FemaleDirCount,PercentWomenDir,Women_dummy,AvgAge,CountRookie,Rookie_dummy,CountNonRookie,CountIndep,PercentIndep,CountNonIndep,CountRookieIndep,CountRookieNonIndep,CountNonRookieIndep,CountNonRookieNonIndep,CountPromoterDir,CountBusyDir,Busy_dummy,AsOnYear,IsCeoMD,HasChairman,HasPromoterOnBoard,IsDualityChairmanMD,IsFamilyManaged,IsFamilyChairman,IsFamilyChairmanAndCEO,CountFirstTermDir,CountTurnOverIndepDir,HasIndepTurnOver,AvgTenureInYearsALL,AvgTenureInYearsIndep,PercentRookieIndep,PercentNonRookieIndep,PercentRookieNonIndep,PercentNonRookieNonIndep,PercentRookie,PercentNonRookie,IsRookieBoard,JustOneRookie,TwoOrMoreRookies,PercentRetiringTermLimit,TermLimitRetireesPcodeList,PercentBusyDir,PercentFirstTermIndep,FirstYearIndepPCodeList,IndepPCodeList,FirstYearPCodeList,PCodeList,CountRetiringTermLimit,CountFirstTermIndep,CountDirTurnOver_exits_retire,PercentDirTurnOver_exits_retire,CountDirTurnOver_real_exits,PercentDirTurnOver_real_exits,CountIndepDirTurnOver_exits_retire,PercentIndepDirTurnOver_exits_retire,CountIndepDirTurnOver_real_exits,PercentIndepDirTurnOver_real_exits,CountDirTurnOver_exits_retire_shifted,PercentIndepDirTurnOver_exits_retire_shifted,CountDirTurnOver_real_exits_shifted,PercentDirTurnOver_real_exits_shifted,CountIndepDirTurnOver_exits_retire_shifted,CountIndepDirTurnOver_real_exits_shifted,PercentIndepDirTurnOver_real_exits_shifted,Company Name,Sales,Profit after tax,PBDITA,Return on total assets,Total assets,Operating profit of non-financial companies,Operating profit of financial companies,Debt to equity ratio (times),Long term borrowings incl current portion,Long term borrowings excl current portion,Debt,Trade payables (Old Sch. VI),Subscribed preference capital,Paid up preference capital (net of forfeited preference capital) (Old Sch. VI),Paid up preference capital (net of forfeited preference capital),Paid up preference shares,Total other receivables,Net cash flow from operating activities,Cash (outflow) due to purchase of fixed assets,Cash inflow due to sale of fixed assets,Cash flow due to purchase of fixed assets,Net cash inflow or (outflow) from investing activities,Research & development expenses,Inter-corporate loans (Old Sch. VI),Deferred tax liability,Net deferred tax liabilities,BVPS,Shares Outstanding,Market Capitalisation,Total Returns,P/B,Market Capitalisation / Enterprise Value,Prowess company code,Entity type,Entity type code,Incorporation year,Industry group,Industry group code,NIC name,NIC code,Ownership group code,Ownership group,Age group,NSE symbol,Head office address,Total Shares (In %) - Shares held,Promoters (In %) - Shares held,Indian Promoters (In %) - Shares held,Foreign Promoters (In %) - Shares held,Non-promoters (In %) - Shares held,Non-promoter Institutions (In %) - Shares held,Non-promoter Non-institutions (In %) - Shares held,BVCommonStock,Office
0,0,0,20MICRONS,2009-03-31,12,2.484907,0,0.000000,0,69.166667,4,0,8,0,0.000000,12,0,4,0,8,0,1,1,2009,0,0,0,0,0,0,0,6,0,0,6.419178,0.000000,0.000000,0.000000,0.333333,0.666667,0.333333,0.666667,0,0,0,0.000000,[],0.000000,0.000000,[],[],['P00032270'],"['P00027662', 'P00027633', 'P00027664', 'P0002...",0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20 Microns Ltd.,1454.8,13.4,144.7,4.38,1210.4,169.5,56.3,1.64,NaN,NaN,586.1,153.6,NaN,NaN,NaN,NaN,1.6,1.4,-94.1,4.2,94.1,-89.9,3.1,NaN,92.4,70.1,25.19,14205248.0,214.50,0.67,0.60,0.28,11.0,Public Ltd.,1.020302e+10,1987.0,Minerals,1.010203e+14,Other mining and quarrying,8.0,2.010104e+10,20 Microns Group,Between 1986 and 1990,20MICRONS,"347, G I D C Industrial Estate, Waghodia, Vad...",100.0,47.21,39.93,7.28,52.79,20.88,31.90,3.578302e+08,Vadodara
1,1,0,20MICRONS,2010-03-31,13,2.564949,0,0.000000,0,69.461538,4,0,9,0,0.000000,13,0,4,0,9,0,1,1,2010,0,0,0,0,0,0,0,7,0,0,6.893994,0.000000,0.000000,0.000000,0.307692,0.692308,0.307692,0.692308,0,0,0,0.000000,[],0.000000,0.000000,[],[],['P00031581'],"['P

In [10]:
def FirDirCity(frame):
    for Symbol in frame['Symbol']:
        # P Code list First year and Total INDEP directors of CURR company in the same city
        pcodecompfirstindep = set(frame.loc[ frame['Symbol'] == Symbol]['FirstYearIndepPCodeList'].explode())
        pcodecomptotindep = set(frame.loc[ frame['Symbol'] == Symbol]['IndepPCodeList'].explode())

        # P Code list First year and Total directors of CURR company
        pcodecompfirst = set(frame.loc[ frame['Symbol'] == Symbol]['FirstYearPCodeList'].explode())
        pcodecomptot = set(frame.loc[ frame['Symbol'] == Symbol]['PCodeList'].explode())

        # P Code list First Year and total INDEP directors of OTHER companies
        pcodeotherfirstindep = frame.loc[ frame['Symbol'] != Symbol]['FirstYearIndepPCodeList'].explode().nunique()
        pcodeothertotindep = frame.loc[ frame['Symbol'] != Symbol]['IndepPCodeList'].explode().nunique()

        # P Code list First Year and total directors of OTHER companies
        pcodeotherfirst = frame.loc[ frame['Symbol'] != Symbol]['FirstYearPCodeList'].explode().nunique()
        pcodeothertot = frame.loc[ frame['Symbol'] != Symbol]['PCodeList'].explode().nunique()
        
        # P Code list First Year and total INDEP directors of OTHER companies excluding CURR company
        pcodeotherfirstindepexcl = sum(1 for x in set(frame.loc[ frame['Symbol'] != Symbol]['FirstYearIndepPCodeList'].explode()) if x not in pcodecompfirstindep)
        
        pcodeothertotindepexcl = sum(1 for x in set(frame.loc[ frame['Symbol'] != Symbol]['IndepPCodeList'].explode()) if x not in pcodecomptotindep)
        
        # P Code list First Year and total directors of OTHER companies excluding CURR company
        pcodeotherfirstexcl = sum(1 for x in set(frame.loc[ frame['Symbol'] != Symbol]['FirstYearPCodeList'].explode()) if x not in pcodecompfirst)
        pcodeothertotexcl = sum(1 for x in set(frame.loc[ frame['Symbol'] != Symbol]['PCodeList'].explode()) if x not in pcodecomptot)
        
        frame.loc[ frame['Symbol'] == Symbol, 'CountOtherFirstYearIndepPCode'] = pcodeotherfirstindep
        frame.loc[ frame['Symbol'] == Symbol, 'CountTotalIndepPCode'] = pcodeothertotindep
        frame.loc[ frame['Symbol'] == Symbol, 'FirstYearDirCityIndep'] = pcodeotherfirstindep / pcodeothertotindep if pcodeothertotindep>0 else 0

        frame.loc[ frame['Symbol'] == Symbol, 'CountOtherFirstYearPCode'] = pcodeotherfirst
        frame.loc[ frame['Symbol'] == Symbol, 'CountTotalPCode'] = pcodeothertot
        frame.loc[ frame['Symbol'] == Symbol, 'FirstYearDirCity'] = pcodeotherfirst / pcodeothertot if pcodeothertot>0 else 0

        frame.loc[ frame['Symbol'] == Symbol, 'CountOtherFirstYearPCodeIndepExcl'] = pcodeotherfirstindepexcl
        frame.loc[ frame['Symbol'] == Symbol, 'CountTotalPCodeIndepExcl'] = pcodeothertotindepexcl
        frame.loc[ frame['Symbol'] == Symbol, 'FirstYearDirCityIndepExcl'] = pcodeotherfirstindepexcl / pcodeothertotindepexcl if pcodeothertotindepexcl>0 else 0
        
        frame.loc[ frame['Symbol'] == Symbol, 'CountOtherFirstYearPCodeExcl'] = pcodeotherfirstexcl
        frame.loc[ frame['Symbol'] == Symbol, 'CountTotalPCodeExcl'] = pcodeothertotexcl
        frame.loc[ frame['Symbol'] == Symbol, 'FirstYearDirCityExcl'] = pcodeotherfirstexcl / pcodeothertotexcl if pcodeothertotexcl>0 else 0

    return frame

In [11]:
cityLevel = firmLoc.groupby(['Office', 'AsOnDate']).progress_apply(FirDirCity)

100%|██████████████████████████████████████████████████████████████████████████████| 1948/1948 [02:34<00:00, 12.65it/s]


In [12]:
cityLevel

Unnamed: 0  FirmID      Symbol    AsOnDate  \
Office        AsOnDate                                                       
Agra          2012-03-31 18734       18734    1771     PAISALO  2012-03-31   
              2013-03-31 18735       18735    1771     PAISALO  2013-03-31   
                         22047       22047    2102     SAMPANN  2013-03-31   
              2014-03-31 18736       18736    1771     PAISALO  2014-03-31   
                         22048       22048    2102     SAMPANN  2014-03-31   
...                                    ...     ...         ...         ...   
West Godavari 2020-03-31 1419         1419     142  ANDHRSUGAR  2020-03-31   
              2021-03-31 1420         1420     142  ANDHRSUGAR  2021-03-31   
              2022-03-31 1421         1421     142  ANDHRSUGAR  2022-03-31   
              2023-03-31 1422         1422     142  ANDHRSUGAR  2023-03-31   
              2024-03-31 1423         1423     142  ANDHRSUGAR  2024-03-31   

                                TotDirCount  LnBoardSize  FemaleDirCount  \
Office        AsOnDate                                                     
Agra          2012-03-31 18734            9     2.197225               0   
              2013-03-31 18735           11     2.397895               0   
                         22047            4     1.386294               0   
              2014-03-31 18736           11     2.397895               0   
                         22048            5     1.609438               0   
...                                     ...          ...             ...   
West Godavari 2020-03-31 1419            16     2.772589               1   
              2021-03-31 1420            16     2.772589               1   
              2022-03-31 1421            17     2.833213               1   
              2023-03-31 1422            17     2.833213               1   
              2024-03-31 1423            19     2.944439               2   

                                PercentWomenDir  Women_dummy     AvgAge  \
Office        AsOnDate                                                    
Agra          2012-03-31 18734         0.000000            0  65.125000   
              2013-03-31 18735         0.000000            0  67.000000   
                         22047         0.000000            0  64.666667   
              2014-03-31 18736         0.000000            0  67.000000   
                         22048         0.000000            0  64.666667   
...                                         ...          ...        ...   
West Godavari 2020-03-31 1419          0.062500            1  79.000000   
              2021-03-31 1420          0.062500            1  79.000000   
              2022-03-31 1421          0.058824            1  79.071429   
              2023-03-31 1422          0.058824            1  79.071429   
              2024-03-31 1423          0.105263            1  77.562500   

                                CountRookie  Rookie_dummy  CountNonRookie  \
Office        AsOnDate                                                      
Agra          2012-03-31 18734            1             0               8   
              2013-03-31 18735            3             1               8   
                         22047            1             0               3   
              2014-03-31 18736            2             1               9   
                         22048            2             0               3   
...                                     ...           ...             ...   
West Godavari 2020-03-31 1419             0             0              16   
              2021-03-31 1420             0             0              16   
              2022-03-31 1421             0             0              17   
              2023-03-31 1422             0             0              17   
              2024-03-31 1423             2             1              17   

                                CountIndep  Percen

In [13]:
def FirDirCitymean(frame):
    frameOut = pd.DataFrame()

    firstYearDirCityIndepMean = frame["FirstYearDirCityIndep"].mean()
    
    firstYearDirCityMean = frame["FirstYearDirCity"].mean()
    
    firstYearDirCityIndepExclMean =  frame["FirstYearDirCityIndepExcl"].mean()
    
    firstYearDirCityExclMean =  frame["FirstYearDirCityExcl"].mean()


    frameOut["FirstYearDirCityIndepMean"] = [firstYearDirCityIndepMean]
    
    frameOut["FirstYearDirCityMean"] = [firstYearDirCityMean]
    
    frameOut["FirstYearDirCityIndepExclMean"] =  [firstYearDirCityIndepExclMean]
    
    frameOut["FirstYearDirCityExclMean"] =  [firstYearDirCityExclMean]

    return frameOut

In [14]:
cityLevelmean = cityLevel.reset_index(drop = True).groupby(['Office', 'AsOnDate']).progress_apply(FirDirCitymean)

100%|████████████████████████████████████████████████████████████████████████████| 1948/1948 [00:01<00:00, 1149.59it/s]


In [15]:
cityLevelmean

FirstYearDirCityIndepMean  FirstYearDirCityMean  \
Office        AsOnDate                                                        
Agra          2012-03-31 0                        0.0                   0.0   
              2013-03-31 0                        1.0                   1.0   
              2014-03-31 0                        1.0                   1.0   
              2015-03-31 0                        1.0                   1.0   
              2016-03-31 0                        0.5                   0.5   
...                                               ...                   ...   
West Godavari 2020-03-31 0                        0.0                   0.0   
              2021-03-31 0                        0.0                   0.0   
              2022-03-31 0                        0.0                   0.0   
              2023-03-31 0                        0.0                   0.0   
              2024-03-31 0                        0.0                   0.0   

                            FirstYearDirCityIndepExclMean  \
Office        AsOnDate                                      
Agra          2012-03-31 0                            0.0   
              2013-03-31 0                            1.0   
              2014-03-31 0                            0.0   
              2015-03-31 0                            1.0   
              2016-03-31 0                            0.0   
...                                                   ...   
West Godavari 2020-03-31 0                            0.0   
              2021-03-31 0                            0.0   
              2022-03-31 0                            0.0   
              2023-03-31 0                            0.0   
              2024-03-31 0                            0.0   

                            FirstYearDirCityExclMean  
Office        AsOnDate                                
Agra          2012-03-31 0                       0.0  
              2013-03-31 0                       1.0  
              2014-03-31 0                       1.0  
              2015-03-31 0                       1.0  
              2016-03-31 0                       0.0  
...                                              ...  
West Godavari 2020-03-31 0                       0.0  
              2021-03-31 0                       0.0  
              2022-03-31 0                       0.0  
              2023-03-31 0                       0.0  
              2024-03-31 0                       0.0  

[1948 rows x 4 columns]

In [16]:
cityLevelmean2 = cityLevelmean.reset_index( drop = False).drop(["level_2"], axis = 1)
cityLevelmean2

,Office,AsOnDate,FirstYearDirCityIndepMean,FirstYearDirCityMean,FirstYearDirCityIndepExclMean,FirstYearDirCityExclMean
0,Agra,2012-03-31,0.0,0.0,0.0,0.0
1,Agra,2013-03-31,1.0,1.0,1.0,1.0
2,Agra,2014-03-31,1.0,1.0,0.0,1.0
3,Agra,2015-03-31,1.0,1.0,1.0,1.0
4,Agra,2016-03-31,0.5,0.5,0.0,0.0
...,...,...,...,...,...,...
1943,West Godavari,2020-03-31,0.0,0.0,0.0,0.0
1944,West Godavari,2021-03-31,0.0,0.0,0.0,0.0
1945,West Godavari,2022-03-31,0.0,0.0,0.0,0.0
1946,West Godavari,2023-03-31,0.0,0.0,0.0,0.0


In [17]:
 cityLevelmean3 = cityLevelmean.reset_index(drop = False).Office.to_frame().reset_index().rename( {"index":"Office Index"}, axis = 1).merge(cityLevelmean2, on = "Office")

In [18]:
cityLevelmean3

,Office Index,Office,AsOnDate,FirstYearDirCityIndepMean,FirstYearDirCityMean,FirstYearDirCityIndepExclMean,FirstYearDirCityExclMean
0,0,Agra,2012-03-31,0.0,0.0,0.0,0.0
1,0,Agra,2013-03-31,1.0,1.0,1.0,1.0
2,0,Agra,2014-03-31,1.0,1.0,0.0,1.0
3,0,Agra,2015-03-31,1.0,1.0,1.0,1.0
4,0,Agra,2016-03-31,0.5,0.5,0.0,0.0
...,...,...,...,...,...,...,...
32483,1947,West Godavari,2020-03-31,0.0,0.0,0.0,0.0
32484,1947,West Godavari,2021-03-31,0.0,0.0,0.0,0.0
32485,1947,West Godavari,2022-03-31,0.0,0.0,0.0,0.0
32486,1947,West Godavari,2023-03-31,0.0,0.0,0.0,0.0


In [19]:
cityLevel.to_csv("Main_Firm_Fin_FirstYearDirCity v160225.csv")

In [20]:
cityLevelmean3.to_csv("Main_City_FirstYearDirCity_Mean v160225.csv")